# 🏆 FanFlow+: A Personalized Sports Fandom Recommendation Engine  
**Kaggle GenAI Intensive Capstone Submission**  

Author: *Quentin G.*  
Date: *April 20, 2025*

---

## 📌 Overview
FanFlow+ is an interactive, GenAI-powered assistant that creates personalized sports team and athlete recommendations based on:
- A user's favorite players or teams  
- Their "fandom vibe" (what draws them in)  
- Their upcoming travel plans (to suggest live events!)  

It combines custom parsing, structured prompting, and markdown output via Google Gemini to simulate a real-world recommender with memory and creativity.

---


## 🧱 Block 1: Setup & Configuration

In [33]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

import os
import re
from IPython.display import Markdown, display
from google.generativeai import GenerativeModel, configure
from google import genai
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

secrets = UserSecretsClient()
api_key = secrets.get_secret("GOOGLE_API_KEY")

genai.configure(api_key=api_key)
genai.__version__


'0.8.4'

## 🧱 Block 2: Exemplar Input (Megan)

In [53]:
friend_name = "Megan"
raw_text = """1. Golden State Warriors, Stephen Curry, Caitlin Clark, Trinity Rodman, Carolina Panthers (ugh)
2. Skill, Hometown Connection, Charisma
3. Lisbon, Portugal | May 10 - May 14
Porto, Portugal | May 14 - May 18
Reykjavik, Iceland | June 28  - July 5
Carolina Beach, NC, USA | September 6 - September 13
Orlando, FL | November 5 - November 11
"""

## 🧱 Block 3: Parsing Utilities

In [54]:
month_map = {
    "jan": "January", "feb": "February", "mar": "March", "apr": "April",
    "may": "May", "jun": "June", "jul": "July", "aug": "August",
    "sep": "September", "sept": "September",
    "oct": "October", "nov": "November", "dec": "December"
}

def normalize_months(text):
    for short, full in month_map.items():
        text = re.sub(rf"\b{short}\b", full, text, flags=re.IGNORECASE)
    return text

## 🧱 Block 4: Parse the User Block

In [55]:
raw_text = raw_text.replace("\n", " ").replace("\r", " ")
raw_text = re.sub(r"\s{2,}", " ", raw_text).strip()

fav = raw_text.split("1.", 1)[-1].split("2.", 1)[0].strip()
vibe = raw_text.split("2.", 1)[-1].split("3.", 1)[0].strip()
travel = raw_text.split("3.", 1)[-1].strip()

travel = re.sub(r"[–\-]+", "–", normalize_months(travel))
travel = re.sub(r"(\d{1,2}–\d{1,2}[a-z]{0,2})\s+(?=[A-Z])", r"\\1\n", travel)
travel = re.sub(r"([a-z]{2,},)\s+(?=[A-Z])", "\n", travel)
travel_lines = [line.strip() for line in travel.splitlines() if line.strip()]

## 🧱 Block 5: Build the Travel Itinerary

In [56]:
# Extract the line starting with "3." for travel
match = re.search(r"3\.\s*(.+)", raw_text)
travel_line = match.group(1).strip() if match else ""

# Normalize formatting
travel_line = re.sub(r"[–—]+", "-", travel_line)
travel_line = re.sub(r"\s*\|\s*", " | ", travel_line)
travel_line = travel_line.strip()

# Parse if valid
travel_itinerary = []
if "|" in travel_line:
    location, dates = travel_line.split("|", 1)
    location = location.strip().title()
    dates = dates.strip().replace("-", "–").title()
    travel_itinerary.append((location, dates))

# Markdown output
itinerary_md = "### 🗺️ Travel Itinerary:\n"
for city, date_range in travel_itinerary:
    itinerary_md += f"- **{city}** — *{date_range}*\n"

print(itinerary_md)

### 🗺️ Travel Itinerary:
- **Lisbon, Portugal** — *May 10 – May 14 Porto, Portugal | May 14 – May 18 Reykjavik, Iceland | June 28 – July 5 Carolina Beach, Nc, Usa | September 6 – September 13 Orlando, Fl | November 5 – November 11*



## 🧱 Block 6: Markdown Summary Profile

In [57]:
travel_context = "\n".join([f"- {city} during {dates}" for city, dates in travel_itinerary])
base_md = f"""## 🧠 Welcome to FanFlow+, {friend_name}!
Here’s your personalized fandom travel profile:

- **Favorite teams/players**: {fav}
- **What draws you to them**: {vibe}
- **Travel itinerary:**
""" + "\n".join([f"  - {city} during {dates}" for city, dates in travel_itinerary])

## 🧱 Block 6a: Add in Few Shot Examples

In [58]:
few_shot_examples = """\
Example recommendation:
- Oklahoma City Thunder (USA) - A rising team with a scrappy underdog mentality and exciting young talent.
- Napoli FC (Italy) - Known for electric energy and a passionate fanbase. They are the Cinderella story of Italian football!
- Detroit Lions (USA) - A classic underdog team on the rise. Their blue-collar identity resonates with fans who root for the resilient.
- Coco Gauff (USA) - Young, bold, and socially aware - she is not just a tennis star, she is a movement.
- Chicago Sky (USA) - A flashy, fun WNBA team with elite talent and growing cultural impact.
"""

## 🧱 Block 7: Gemini Prompt Engineering (Few-Shot + Markdown Output)

In [59]:
prompt = f"""
You are a helpful sports fandom assistant. Do not assume current rankings or coaches unless explicitly told in the prompt. Please avoid outdated references such as Bill Belichick coaching the Patriots or Tom Brady still playing.
Based on the following user preferences, suggest **5 new teams or athletes** the user might enjoy following. Include a short, engaging reason for each.

**User's favorite teams/players:** {fav}  
**Fandom vibe (if any):** {vibe}  

**Travel plans include:**  
{travel_context}

---

If any of the recommended teams are playing in these locations during those dates, highlight the game(s) they could attend 🎟️.  
Mention:
- Where the game is  
- What league it's part of  
- Why it would be fun or meaningful for the user 🌟  

If no relevant games match, still suggest **at least one live sporting event** 🏟️ in each city during those dates that matches the user's vibe or interests.  
Include:
- Stadium
- League
- Why it may still be worth checking out 💡

Please:
- Include the **city and country** with each team name
- Interpret abbreviations (e.g. "Cards" → "Cardinals") using sport/location
- Use engaging language and emoji

---

{few_shot_examples}

---

Return the result as a **Markdown-formatted bullet list** only.
"""


## 🧱 Block 8: Run Gemini Model

In [60]:
model = GenerativeModel(model_name="models/gemini-2.0-flash-exp")
chat = model.start_chat()
response = chat.send_message(prompt)

## 🧱 Block 9: Output + Reflection

In [61]:
print("=== TRAVEL ITINERARY DEBUG ===")
for c, d in travel_itinerary:
    print(f"→ {c} during {d}")
    
final_md = base_md + f"""

---

### 🔮 Gemini Recommendations:
{response.text}

---

### 💭 Project Reflection:
This project demonstrates:
- Prompt engineering with few-shot examples
- Structured output
- LLM output formatting into markdown
- Personalization based on sports fandom and travel plans
- Agent-style follow-up loop (optional)
- Simulation of retrieval behavior using generative context (events by city/date)
- Prompt disambiguation and fan terminology mapping

Limitations:
- Gemini responses are generative and not based on live sports data.
- Some disambiguation still depends on how the LLM interprets vague team names.

Next steps could include:
- Embedding or vector search based on fandom profiles
- Real-time event scraping or integration with an API like ESPN or Ticketmaster
- Packaging into a shareable web app (e.g. Streamlit or Flask)
"""

display(Markdown(final_md))

=== TRAVEL ITINERARY DEBUG ===
→ Lisbon, Portugal during May 10 – May 14 Porto, Portugal | May 14 – May 18 Reykjavik, Iceland | June 28 – July 5 Carolina Beach, Nc, Usa | September 6 – September 13 Orlando, Fl | November 5 – November 11


## 🧠 Welcome to FanFlow+, Megan!
Here’s your personalized fandom travel profile:

- **Favorite teams/players**: Golden State Warriors, Stephen Curry, Caitlin Clark, Trinity Rodman, Carolina Panthers (ugh)
- **What draws you to them**: Skill, Hometown Connection, Charisma
- **Travel itinerary:**
  - Lisbon, Portugal during May 10 – May 14 Porto, Portugal | May 14 – May 18 Reykjavik, Iceland | June 28 – July 5 Carolina Beach, Nc, Usa | September 6 – September 13 Orlando, Fl | November 5 – November 11

---

### 🔮 Gemini Recommendations:
Here are five new teams/athletes you might enjoy following, based on your preferences:

*   **SL Benfica (Lisbon, Portugal)** - One of Portugal's "Big Three" football clubs, known for their passionate fanbase and historic success. Experience the electric atmosphere of Portuguese football!

    *   🏟️ **Live Sporting Event:** Consider attending a Liga Portugal match at the *Estádio da Luz* if SL Benfica has a home game during your visit. Even if they aren't playing, the stadium tour is a must for any football fan! 💡

*   **FC Porto (Porto, Portugal)** - Another of Portugal's "Big Three," Porto boasts a rich history and intense rivalry with Benfica. Their matches are full of skill and passion.

*   **Icelandic Women's National Football Team (Iceland)** - With a small population and incredible passion for the sport, Iceland’s women's team is a true underdog story. They consistently punch above their weight on the international stage.

*   **Los Angeles Chargers (USA)** - If you're sticking with the NFL, the Chargers offer exciting quarterback play and a flashy offensive style, reminiscent of the Warriors' fast-paced action.

*   **Atlanta Dream (USA)** - Following Caitlin Clark might have sparked an interest in the WNBA. The Dream are a young, exciting team with a rising star in Rhyne Howard, playing with a similar level of skill and fire 🔥

    *   🏟️ **Live Sporting Event in Orlando, FL:** Consider catching the Orlando Magic (NBA) at the Amway Center during your September trip 🏀 They have a young, exciting team. Even if it's preseason, the energy is palpable! 💡


---

### 💭 Project Reflection:
This project demonstrates:
- Prompt engineering with few-shot examples
- Structured output
- LLM output formatting into markdown
- Personalization based on sports fandom and travel plans
- Agent-style follow-up loop (optional)
- Simulation of retrieval behavior using generative context (events by city/date)
- Prompt disambiguation and fan terminology mapping

Limitations:
- Gemini responses are generative and not based on live sports data.
- Some disambiguation still depends on how the LLM interprets vague team names.

Next steps could include:
- Embedding or vector search based on fandom profiles
- Real-time event scraping or integration with an API like ESPN or Ticketmaster
- Packaging into a shareable web app (e.g. Streamlit or Flask)


## 🧪 Bonus: Try Your Own FanFlow+ Submission

In [44]:
# --- User Input Prompt ---
print("""Paste your full input using the format below. Press Enter twice to finish.
""")
print("""1. Who are your favorite teams or athletes?
   Example: Warriors, Kobe Bryant, Serena Williams

2. What makes you root for someone? (e.g., underdog, dominant, socially impactful)

3. Where are you traveling and when?
   Format: City, Country Date Range
   Example:
   Barcelona, Spain June 30 - July 4
   Amsterdam, Netherlands July 4 - July 13
""")

friend_name = input("What’s your name? ").strip()
print("Paste below 👇")

raw_lines = []
while True:
    line = input()
    if line.strip() == "":
        break
    raw_lines.append(line.strip())

raw_text = " ".join(raw_lines)

Paste your full input using the format below. Press Enter twice to finish.

1. Who are your favorite teams or athletes?
   Example: Warriors, Kobe Bryant, Serena Williams

2. What makes you root for someone? (e.g., underdog, dominant, socially impactful)

3. Where are you traveling and when?
   Format: City, Country Date Range
   Example:
   Barcelona, Spain June 30 - July 4
   Amsterdam, Netherlands July 4 - July 13



What’s your name?  Justin


Paste below 👇


 Rafa Nadal, Gabby Douglas, the Atlanta Braves, the Chicago Bears
 Underdog, dynasty
 Buenos Aires, Argentina Sep 12- 26
 Osaka, Japan Dec 25 - 31
 


Now rerun **Blocks 3 through 9** to parse your data, send it to Gemini, and get a personalized fandom profile!

### 🔁 Rerun Instructions

Once you've entered your custom info above:

➡️ Rerun the following blocks to see your personalized recommendations:
- **Block 3**: Parsing Utilities  
- **Block 4**: Parse the User Block  
- **Block 5**: Build the Travel Itinerary  
- **Block 6**: Markdown Summary Profile  
- **Block 7**: Gemini Prompt  
- **Block 8**: Run Gemini Model  
- **Block 9**: Output + Reflection

In Jupyter or Colab:
- You can highlight all cells below this one and press **Shift+Enter**  
- Or click **"Runtime → Run after"** from the menu (in Colab)
